In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta

# Load data
csv_file_path = "/Users/balmeru/Downloads/median_fyrchange_dates.csv"
df = pd.read_csv(csv_file_path)
df['rdq'] = pd.to_datetime(df['rdq'], errors='coerce')

df['txdbq'] = df['txdbq'].fillna(0)
df['dvpq'] = df['dvpq'].fillna(0)

df.loc[:, "income_good"] = False
df.loc[~df["ibq"].isna(), "income_good"] = True

#1
df.loc[:, "se1_good"] = False
df.loc[~df["teqq"].isna(), "se1_good"] = True

#2
df.loc[:, "se2_good"] = False
df.loc[~df["ceqq"].isna() & ~df["pstkq"].isna(), "se2_good"] = True

#3
df.loc[:, "se3_good"] = False
df.loc[~df["atq"].isna() & ~df["ltq"].isna(), "se3_good"] = True


# Filter out cases where fyr_change is False and conditions for income_good and se goods are met
df_filtered = df[(df['fyr_change_dummy'] == 0) & (df['income_good'] == True) & ((df['se1_good'] == True) | (df['se2_good'] == True) | (df['se3_good'] == True))]
target_year = 1988
reference_year = 1987
expected_dates = []
for tic in df_filtered['tic'].unique():
    tic_data = df_filtered[
        (df_filtered['tic'] == tic) & 
        (df_filtered['fyearq'] == reference_year)
    ]  
    for quarter in range(1, 5):
        quarter_data = tic_data[
            (tic_data['fqtr'] == quarter)
        ]
        
        if not quarter_data.empty:
            reference_date = quarter_data['rdq'].iloc[0]
            expected_date = reference_date + timedelta(weeks=52)
        else:
            expected_date = np.nan  
        
        expected_dates.append({
            'quarter': f"1988Q{quarter}",  
            'tic': tic,                   
            'expected_announcement_date': expected_date  
        })
expected_dates_df = pd.DataFrame(expected_dates)
expected_dates_df['quarter'] = pd.Categorical(expected_dates_df['quarter'], categories=['1988Q1', '1988Q2', '1988Q3', '1988Q4'], ordered=True)

pivot_table = expected_dates_df.pivot_table(
    index='quarter',   
    columns='tic',    
    values='expected_announcement_date'  
)

pivot_table_cleaned = pivot_table.dropna(axis=1, how='all')
print("Pivot Table corrected for both early and late reporting:")
print(pivot_table_cleaned)



Pivot Table corrected for both early and late reporting:
tic          0015B      0033A      0053A      0067A      0097A      0131A  \
quarter                                                                     
1988Q1  1988-03-02 1988-04-28 1988-05-11        NaT 1988-05-25 1988-04-19   
1988Q2  1988-06-01 1988-07-28 1988-08-05        NaT 1988-07-15 1988-08-01   
1988Q3  1988-09-06 1988-10-31 1988-11-07 1988-04-20 1988-10-19 1988-10-24   
1988Q4  1989-01-17 1989-02-20 1989-02-22        NaT 1989-03-14 1989-03-13   

tic          0139A      0141A      0173A      0189A  ...       ZION  \
quarter                                              ...              
1988Q1  1988-04-19 1988-04-21 1988-08-17        NaT  ... 1988-04-19   
1988Q2  1988-07-25 1988-07-26 1988-11-22        NaT  ... 1988-07-21   
1988Q3  1988-10-24 1988-10-25 1989-02-15 1988-11-10  ... 1988-10-18   
1988Q4  1989-02-02 1989-01-24 1989-05-31 1989-02-21  ... 1989-01-31   

tic            ZMX        ZNT     ZNTL.1       ZOND  

In [2]:
pivot_table_cleaned.to_csv('expected_dates_1988.csv', index=True)